In [ ]:
!pip install selenium
!apt-get update
!apt-get install firefox -y
!pip install selenium
# Download and install GeckoDriver
!wget https://github.com/mozilla/geckodriver/releases/download/v0.30.0/geckodriver-v0.30.0-linux64.tar.gz
!tar -xvzf geckodriver-v0.30.0-linux64.tar.gz
!chmod +x geckodriver
!mv geckodriver /usr/local/bin/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:8 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,781 kB]
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu fo

In [ ]:
import pandas as pd
import gdown
import re
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import os
from google.colab import drive
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import gc
import time
import pytz
import datetime

In [ ]:
gdown.download(id = '1DuT3nnIouLKuRFbGo5lqcQwKTTb5GM8r')
df = pd.read_csv('/content/suburbs.csv')

Downloading...
From: https://drive.google.com/uc?id=1DuT3nnIouLKuRFbGo5lqcQwKTTb5GM8r
To: /content/suburbs.csv
100%|██████████| 975k/975k [00:00<00:00, 127MB/s]


In [ ]:
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Files-Required/client project')

Mounted at /content/drive


In [ ]:
df.shape

(15879, 2)

In [ ]:
def driver_setup():# Assigning a Headless Firefox Driver
  options = webdriver.FirefoxOptions()
  options.add_argument('--headless')
  options.add_argument('--no-sandbox')
  options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Firefox(options=options)
  return driver

df_urbdat = pd.DataFrame()
max_rows_in_memory = 10
suburb_data_list = []  # list to collect row data for a suburb.
temp_file = 'temp_data.csv' # planning an effecient method to store all the data in disk.

for i in range(20): # Suburb Range
  print(i) # to understand the compleation rate.

  url = 'https://www.domain.com.au/suburb-profile/'+df['0'][i]
  driver = driver_setup() # for each link we are setting a new Firefox instance
  driver.get(url)
  html = driver.page_source
  soup = BeautifulSoup(html, 'html.parser')
  table = soup.find("table")

  try:  # if there is no table, the entire process is stopped.
    rows = table.find_all("tr")[1:]  # finding all the rows in the table to access one by one later.

  # try:# if there is no table the entire process id stoped.
  #   rows = table.find_all("tr")[1:] # finding all the rows in the table to access one by one later.
  #   tempdf = pd.DataFrame() # empty dataframe to collect rows of data for a suburb.

    for row in range(len(rows)): # accessing the rows.
        cells = rows[row].find_all("td") # data collwxtion process collecing all cell data
        try: # if the data is avilable it will collect or ignore.
          bedrooms = cells[0].text.strip()
        except:
          bedrooms = None
        try:
          prop_type = cells[1].text.strip()
        except:
          prop_type = None
        try:
          median_price = cells[2].text.strip()
        except:
          median_price = None
        try:
          days_on_market = cells[3].text.strip()
        except:
          days_on_market = None
        try:
          clearance_rate = cells[4].text.strip()
        except:
          clearance_rate = None
        try:
          sold_this_year = cells[5].text.strip()
        except:
          sold_this_year = None
        try:
          population = soup.find('div', {'class': 'css-48zwbo'}).find('div', {'class': 'css-54bw0x'}).text
        except:
          population = None
        try:
          average_age = soup.find_all('div', {'class': 'css-48zwbo'})[1].find('div', {'class': 'css-54bw0x'}).text
        except:
          average_age = None
        try:
          owner_percent = soup.find('span', {'data-testid': 'left-value'}).text.strip()
        except:
          owner_percent = None
        try:
          renter_percent = soup.find('span', {'data-testid': 'right-value'}).text.strip()
        except:
          renter_percent = None
        try:
          family_percent = soup.find_all('span', {'data-testid': 'left-value'})[1].text.strip()
        except:
          family_percent = None
        try:
          single_percent = soup.find_all('span', {'data-testid': 'right-value'})[1].text.strip()
        except:
          single_percent = None

        # Set the timestamp in Mosman, NSW, Australia (GMT+10)
        australia_timezone = pytz.timezone('Australia/Sydney')
        current_time = datetime.datetime.now(australia_timezone)

        # single row collection, we also have the buttonfunction and it has more features like Rental median price.

        row_data = { # features to dictionary.
            "Bedrooms": bedrooms,
            "Type": prop_type,
            "Median Price": median_price,
            "Avg Days on Market": days_on_market,
            "Clearance Rate": clearance_rate,
            "Sold This Year": sold_this_year,
            'POPULATION': population,
            'AVERAGE AGE': average_age,
            'OWNER': owner_percent,
            'RENTER': renter_percent,
            'FAMILY': family_percent,
            'SINGLE': single_percent,
            'SUBURB': df['0'][i],
            'Time_stamp': current_time
        }

        one_row_data = pd.DataFrame(row_data,index=[0]) # dictionary to one row dataframe.

        try: # if there is no row data then the entire loop has nothing to do.

          # row's button function.
          button = driver.find_element(By.XPATH, '//*[@id="__next"]/section/section[4]/div/div/div[2]/table/tbody['+str(row)+']/tr/td[7]/button') # finding the row's button.
          button.click()
          button_html = button.get_attribute('outerHTML')
          soup = BeautifulSoup(button_html, 'html.parser')
          title_element = soup.find('div').find('title')
          title = title_element.get_text() # button open or close checker.

          html = driver.page_source
          soup = BeautifulSoup(html, 'html.parser')

          tr_element = soup.find('tr', class_='css-1wpy7ho')
          div_elements = tr_element.find_all('div', class_='css-15ydh5a')

          datad = {}

          # Extract the data from each div element and add it to the dictionary
          for div in div_elements:
              h4_element = div.find('h4', class_='css-srsjf4')
              data_point = div.find('div', class_='css-1s6j0do')
              label = h4_element.text.strip(':')
              value = data_point.text.strip()
              datad[label] = value

          tdf = pd.DataFrame(datad, index=[0]) # dataframe to collect the data from the trend.

          table_rows = soup.find_all('tr')  # table of one rows trend year wise.
          data = {
              'Year': [],
              'Median Price': [],
              'Growth': [],
              'Number of Sales': []
          }
          for row in table_rows[1:]:
              columns = row.find_all('td')
              if len(columns) == 4:
                  year = columns[0].text.strip()
                  median_price = columns[1].text.strip()
                  growth = columns[2].text.strip()
                  num_sales = columns[3].text.strip()

                  data['Year'].append(year)
                  data['Median Price'].append(median_price)
                  data['Growth'].append(growth)
                  data['Number of Sales'].append(num_sales)

          yearly_df = pd.DataFrame(data)

          years = yearly_df['Year'].tolist()
          for year in years:
              tdf.loc[0, f'median price {year}'] = yearly_df.loc[yearly_df['Year'] == year, 'Median Price'].item()
              tdf.loc[0, f'growth rate {year}'] = yearly_df.loc[yearly_df['Year'] == year, 'Growth'].item()
              tdf.loc[0, f'no of sales {year}'] = yearly_df.loc[yearly_df['Year'] == year, 'Number of Sales'].item()

          one_row_data = pd.concat([one_row_data,tdf], axis=1) # appending all the columns of new dataframe in the one_row_data.

        except:
          continue
        suburb_data_list.append(one_row_data.to_dict(orient='records')[0])  # appending row data to the suburb_data_list


        # Effecient block to save the code direct to disk.

        if len(suburb_data_list) >= max_rows_in_memory: # Save the current portion of data to disk
          temp_df = pd.DataFrame(suburb_data_list)

          if os.path.isfile(temp_file):
            temp_df_existing = pd.read_csv(temp_file)
            existing_columns = set(temp_df_existing.columns)

            temp_df = pd.DataFrame(suburb_data_list)
            new_columns = set(temp_df.columns)

            if existing_columns != new_columns:
              # Update the header with new columns
              temp_df.to_csv(temp_file, mode='w', header=True, index=False)
            else:
              # Append to the existing temporary file without writing the header
              temp_df.to_csv(temp_file, mode='a', header=False, index=False)
          else:
            # Create a new temporary file and write the data with the header
            temp_df = pd.DataFrame(suburb_data_list)
            temp_df.to_csv(temp_file, mode='w', header=True, index=False)

          suburb_data_list = []

  except:# no table block
    None

  driver.quit() # killing the firefox instance to maintain the speed of data sourcing.
  gc.collect()
  os.system("taskkill /f /im firefox.exe")

if suburb_data_list:
  temp_df = pd.DataFrame(suburb_data_list)
  temp_df.to_csv('temp_data.csv', mode='a', header=False, index=False)

df_urbdat = pd.concat(pd.read_csv('temp_data.csv', chunksize=max_rows_in_memory), ignore_index=True)
os.remove('temp_data.csv')


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [ ]:
df_urbdat

,Bedrooms,Type,Median Price,Avg Days on Market,Clearance Rate,Sold This Year,POPULATION,AVERAGE AGE,OWNER,RENTER,...,no of sales 2022,median price 2021,growth rate 2021,no of sales 2021,median price 2020,growth rate 2020,no of sales 2020,median price 2019,growth rate 2019,no of sales 2019
0,3,House,$570k,51 days,-,16,"3,103",20 to 39,63%,37%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,House,$635k,37 days,-,70,"3,103",20 to 39,63%,37%,...,21.0,$400k,-2%,18.0,$408k,10.3%,14.0,$370k,-3.9%,13.0
2,5,House,$735k,-,-,11,"3,103",20 to 39,63%,37%,...,116.0,$470k,7.1%,99.0,$439k,-1.1%,66.0,$444k,-5.9%,54.0
3,3,House,$428k,61 days,-,112,"9,701",60+,68%,32%,...,30.0,$239k,13.8%,25.0,$210k,1%,27.0,$208k,18.9%,33.0
4,4,House,$590k,79 days,-,48,"9,701",60+,68%,32%,...,166.0,$284k,9.2%,158.0,$260k,12.1%,133.0,$232k,0.9%,180.0
5,5,House,-,-,-,9,"9,701",60+,68%,32%,...,71.0,$410k,20.9%,73.0,$339k,1.8%,77.0,$333k,2.5%,78.0
6,2,Unit,$280k,84 days,-,15,"9,701",60+,68%,32%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,3,Unit,-,-,-,3,"9,701",60+,68%,32%,...,18.0,$190k,-5%,16.0,$200k,26.6%,25.0,$158k,-,18.0
8,3,House,-,-,-,9,869,60+,85%,15%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,4,House,-,-,-,4,869,60+,85%,15%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_urbdat.to_csv('df_urbdat1.csv', index=False)